# Figure 1a. Load summary statistics

In [12]:
from enigmatoolbox.datasets import load_summary_stats

# Load summary statistics for a given disease (e.g., epilepsy)
sum_stats = load_summary_stats('epilepsy')

# List available summary statistic tables
for table_name in sum_stats:
    print(table_name)
    
# Extract Cohen's d values
CT_d = sum_stats['CortThick_case_vs_controls_ltle']['d_icv']
SV_d = sum_stats['SubVol_case_vs_controls_ltle']['d_icv']


CortThick_case_vs_controls_allepilepsy
SubVol_case_vs_controls_allepilepsy
CortThick_case_vs_controls_gge
SubVol_case_vs_controls_gge
CortThick_case_vs_controls_ltle
SubVol_case_vs_controls_ltle
CortThick_case_vs_controls_rtle
SubVol_case_vs_controls_rtle


,CorticalThickness,Structure,d_icv,se_icv,low_ci_icv,up_ci_icv,n_controls,n_patients,pobs,fdr_p
0,TLEMTS-L_vs_CTL,L_bankssts,-0.100,0.112,-0.319,0.119,1279,383,3.720000e-01,3.720000e-01
1,TLEMTS-L_vs_CTL,L_caudalanteriorcingulate,0.181,0.060,0.064,0.299,1343,411,3.000000e-03,3.000000e-03
2,TLEMTS-L_vs_CTL,L_caudalmiddlefrontal,-0.403,0.070,-0.539,-0.266,1344,412,7.070000e-09,7.070000e-09
3,TLEMTS-L_vs_CTL,L_cuneus,-0.190,0.127,-0.438,0.059,1342,412,1.340000e-01,1.340000e-01
4,TLEMTS-L_vs_CTL,L_entorhinal,-0.445,0.072,-0.586,-0.303,1102,303,7.350000e-10,7.350000e-10
...,...,...,...,...,...,...,...,...,...,...
63,TLEMTS-L_vs_CTL,R_superiorparietal,-0.474,0.160,-0.787,-0.161,1347,412,3.000000e-03,3.000000e-03
64,TLEMTS-L_vs_CTL,R_superiortemporal,-0.165,0.061,-0.285,-0.044,1249,398,7.000000e-03,7.000000e-03
65,TLEMTS-L_vs_CTL,R_supramarginal,-0.303,0.089,-0.477,-0.129,1292,400,1.000000e-03,1.000000e-03
66,TLEMTS-L_vs_CTL,R_temporalpole,-0.033,0.060,-0.150,0.085,1344,410,5.860000e-01,5.860000e-01


# Figure 2a. Surface data visualization

In [8]:
from enigmatoolbox.utils.useful import reorder_sctx
import numpy as np
from enigmatoolbox.utils.parcellation import parcel_to_surface
from enigmatoolbox.plotting import plot_cortical, plot_subcortical

# Map parcellated data to the surface (cortical values only)
CT_d_fsa5 = parcel_to_surface(CT_d,'aparc_fsa5')

# Project data to the surface templates
plot_cortical(array_name=CT_d_fsa5, surface_name="fsa5",
              size=(800, 400), cmap='Blues_r', color_bar=True, color_range=(-0.5, 0.5))

plot_subcortical(array_name=SV_d, size=(800, 400),
                 cmap='Blues_r', color_bar=True, color_range=(-0.5, 0.5))


# Figure 3a. Fetch disease-related gene expression data

In [ ]:
from enigmatoolbox.datasets import fetch_ahba, risk_genes

# Fetch gene expression data
genes = fetch_ahba()

# Get the names of epilepsy-related genes (Focal HS phenotype)
epilepsy_genes = risk_genes('epilepsy')['focalhs']

# Extract gene expression data for epilepsy (Focal HS)
epilepsy_gene_data = genes[genes.columns.intersection(epilepsy_genes)]


# Figure 4a. Load connectivity data

In [ ]:
from enigmatoolbox.datasets import load_fc, load_sc

# Load functional connectivity data
fc_ctx, fc_ctx_labels, fc_sctx, fc_sctx_labels = load_fc()

# Load structural connectivity data
sc_ctx, sc_ctx_labels, sc_sctx, sc_sctx_labels = load_sc()


# Figure 5a. Hub susceptibility model

In [ ]:
import numpy as np
from enigmatoolbox.permutation_testing import spin_test, shuf_test

# Remove subcortical values corresponding to the ventricles
SubVol_Z_LTLE_r_mean_noVent = SubVol_Z_LTLE_r_mean.drop(['LLatVent', 'RLatVent'])

# Compute weighted degree centrality measures
fc_ctx_dc = np.sum(fc_ctx, axis=0)
fc_sctx_dc = np.sum(fc_sctx, axis=1)

# Perform spatial correlations between hubs and mean atrophy
fc_ctx_r = np.corrcoef(fc_ctx_dc, CortThick_Z_LTLE_mean)[0, 1]
fc_sctx_r = np.corrcoef(fc_sctx_dc, SubVol_Z_LTLE_r_mean_noVent)[0, 1]

# Spin permutation testing for two cortical maps
fc_ctx_p = spin_test(fc_ctx_dc, CortThick_Z_LTLE_mean, surface_name='fsa5',
                     n_rot=1000, type='pearson')

# Shuf permutation testing for two subcortical maps
fc_sctx_p = shuf_test(fc_sctx_dc, SubVol_Z_LTLE_r_mean_noVent,
                      n_rot=1000, type='pearson')


# Figure 6a. Disease epicenter mapping

In [ ]:
import numpy as np
from enigmatoolbox.permutation_testing import spin_test

# Identify functional epicenters
fc_ctx_epi = []
fc_ctx_epi_p = []
for seed in range(fc_ctx.shape[0]):
    seed_con = fc_ctx[:, seed]
    fc_ctx_epi = np.append(fc_ctx_epi, np.corrcoef(seed_con, CortThick_Z_LTLE_mean)[0, 1])
    fc_ctx_epi_p = spin_test(seed_con, CortThick_Z_LTLE_mean, surface_name='fsa5',
                         n_rot=1000, type='pearson')

fc_sctx_epi = []
fc_sctx_epi_p = []
for seed in range(fc_sctx.shape[0]):
    seed_con = fc_sctx[seed, :]
    fc_sctx_epi = np.append(fc_sctx_epi, np.corrcoef(seed_con, CortThick_Z_LTLE_mean)[0, 1])
    fc_sctx_epi_p = spin_test(seed_con, CortThick_Z_LTLE_mean, surface_name='fsa5',
                             n_rot=1000, type='pearson')
    